<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Get profile connections
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_connections_from_network.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #network #connections #naas_drivers #analytics #csv #html #image #content #plotly

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

This notebook get your LinkedIn profile connections and save it into a CSV file. 

## Input

### Get common variables, functions

In [6]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

## Model

### Get connections

In [9]:
df_connections = get_data(LK_PROFILE_CONNECTIONS)
print('✅ Initial connections fetched:', len(df_connections))
df_connections.head(1)

✅ Initial connections fetched: 9228


,FIRSTNAME,LASTNAME,OCCUPATION,CREATED_AT,PROFILE_URL,PROFILE_PICTURE,PROFILE_ID,PUBLIC_ID,DATE_EXTRACT,OWNER_ID,OWNER_NAME
0,Gerry,VanderStel,Principal Data Analyst at Medtronic,2022-06-25 00:29:29,https://www.linkedin.com/in/ACoAAA7tG_cBPxSxwX...,https://media-exp2.licdn.com/dms/image/C5603AQ...,ACoAAA7tG_cBPxSxwXLTOqMmr88tYF-6qd7Pptc,gerry-vanderstel-133a476b,2022-06-25 11:09:04,ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg,Jérémy Ravenel


### Update connections from LinkedIn network

In [4]:
def update_connections(df, key=None, profile_id=None, fullname=None, min_updated_time=300):
    # Init output
    df_update = pd.DataFrame()
    
    # If df is empty then get all data
    if len(df) > 0:
        if "DATE_EXTRACT" in df.columns:
            last_update_date = df["DATE_EXTRACT"].max()
            time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
            minute_last_update = time_last_update.total_seconds() / 60
            if minute_last_update > min_updated_time:
                # If dataframe not empty, get last connections
                profiles = df[key].unique()
                start = 0
                count = 100
                while True:
                    tmp_new = linkedin.connect(LI_AT, JSESSIONID).network.get_connections(start=start, limit=count)
                    # Check if existing profile in each call
                    tmp_new = tmp_new[~tmp_new.PROFILE_ID.isin(profiles)]
                    df_update = pd.concat([df_update, tmp_new])
                    if len(tmp_new) == 0:
                        break

                    # Get more profile
                    start += count
                print(f"-> New connections fetched: {len(df_update)}.")
            else:
                print(f"🛑 Nothing to update. Last update done {int(minute_last_update)} minutes ago.")
    else:
        df_update = linkedin.connect(LI_AT, JSESSIONID).network.get_connections(limit=-1)
        
    # Concat and add extract date
    df = pd.concat([df_update, df]).drop_duplicates(key, keep="first")
    df.loc[:, 'OWNER_ID'] = profile_id
    df.loc[:, 'OWNER_NAME'] = fullname
    print(f"✅ {len(df)} connections fetched.")
    return df.reset_index(drop=True)
    
df_update = update_connections(df_connections,
                               key="PROFILE_ID",
                               profile_id=LK_PROFILE_ID,
                               fullname=LK_FULLNAME)
df_update.head(1)

-> New connections fetched: 0.
✅ 9228 connections fetched.


,FIRSTNAME,LASTNAME,OCCUPATION,CREATED_AT,PROFILE_URL,PROFILE_PICTURE,PROFILE_ID,PUBLIC_ID,DATE_EXTRACT,OWNER_ID,OWNER_NAME
0,Gerry,VanderStel,Principal Data Analyst at Medtronic,2022-06-25 00:29:29,https://www.linkedin.com/in/ACoAAA7tG_cBPxSxwX...,https://media-exp2.licdn.com/dms/image/C5603AQ...,ACoAAA7tG_cBPxSxwXLTOqMmr88tYF-6qd7Pptc,gerry-vanderstel-133a476b,2022-06-25 11:09:04,ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg,Jérémy Ravenel


## Output

### Save dataframe

In [5]:
save_data(df_update, LK_PROFILE_CONNECTIONS)

👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
✅ Dataframe successfully saved in CSV: LinkedIn/Inputs/LINKEDIN_PROFILE_CONNECTIONS_ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg.csv
